In [ ]:
import fitz
import io
from PIL import Image
from unstructured.partition.pdf import partition_pdf

def get_text(file_path):
    text = ''
    try:
        blocks = partition_pdf(filename=file_path)
        text = [str(el) for el in blocks]
    except ImportError:
        print('Missing required dependencies')
    except Exception as e:
        print(f'Error encountered while loading file: {e}')
    
    return text


def get_images(file_path):
    doc = fitz.open(file_path)
    images_dict = {}
    for page_index in range(len(doc)):
        page = doc.load_page(page_index)
        image_li = page.get_images(full=True)
        for img_idx, img_info in enumerate(image_li):
            # image id
            xref = img_info[0]

            # get base img using id
            img = doc.extract_image(xref)
            img_bytes = img['image']
            img_ext = img['ext']

            try:
                image = Image.open(io.BytesIO(img_bytes))
                image_filename = f"{file_path}page{page_index+1}_img{img_idx+1}.{img_ext}"
                images_dict[image_filename] = image
            except Exception as e:
                print(f'could not load image. warning {e}')
    
    return images_dict


def extract_text_and_image(file_path):
    text = get_text(file_path)
    images = get_images(file_path)
    return {'text': text, 'images': images}
    

info = extract_text_and_image('data/res_1.pdf')

In [110]:
for txt in info['text']:
    print(txt, '\n')

FUNCTIONAL (EXPERIENCED) 

IM A. SAMPLE I 1234 North 55 Street Bellevue, Nebraska 68005 (402) 292-2345 imasample1@xxx.com 

SUMMARY OF QUALIFICATIONS 

Exceptionally well organized and resourceful Professional with more than six years experience and a solid academic background in accounting and financial management; excellent analytical and problem solving skills; able to handle multiple projects while producing high quality work in a fast-paced, deadline-oriented environment. 

EDUCATION 

Bachelor of Science, Bellevue University, Bellevue, NE (In Progress) 

Major: Accounting Expected Graduation Date: January, 20xx 

Minor: Computer Information Systems GPA to date: 3.95/4.00 

PROFESSIONAL ACCOMPLISHMENTS 

Accounting and Financial Management  Developed and maintained accounting records for up to fifty bank accounts.  Formulated monthly and year-end financial statements and generated various payroll records, including federal and state payroll reports, annual tax reports, W-2 and 1